In [1]:
print("Ok")


Ok


In [23]:
%pwd

'd:\\chatbot\\medicalchatbot'

In [3]:
import os
os.chdir("../")

In [24]:
%pwd

'd:\\chatbot\\medicalchatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [7]:

extracted_data=load_pdf_file(data='Data/')

In [8]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 2345


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

C:\Users\gaswi\AppData\Local\Temp\ipykernel_14120\4043855499.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
d:\chatbot\medicalchatbot\medibot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [17]:
from dotenv import load_dotenv
load_dotenv()

True

In [18]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GEMINI_API_KEY=os.environ.get('GEMINI_API_KEY')


In [29]:
from pinecone import Pinecone, ServerlessSpec
import os

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
)


In [30]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [31]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [32]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [33]:
docsearch

In [34]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [35]:
retrieved_docs = retriever.invoke("What is Acne?")

In [36]:
retrieved_docs

[Document(id='1ac7fd50-bdd3-4d99-ba63-0671a38811c9', metadata={'author': 'Tierney, Lawrence M.; Saint, Sanjay.; Whooley, Mary A.', 'creationdate': '2010-12-22T13:15:56+00:00', 'creator': 'PyPDF', 'keywords': '0071637907\r\n9780071637909', 'moddate': '2018-08-02T09:28:12+05:30', 'page': 412.0, 'page_label': '400', 'producer': 'PyPDF', 'source': 'Data\\Current Essentials of Medicine.pdf', 'subject': 'McGraw-Hill Medical', 'title': 'Current Essentials of Medicine', 'total_pages': 607.0}, page_content='400 Current Essentials of Medicine\nAcne Vulgaris\n■ Essentials of Diagnosis\n• Often occurs at puberty, though onset may be delayed until the\nthird or fourth decade\n• Open and closed comedones the hallmarks\n• Severity varies from comedone to papular or pustular inﬂamma-\ntory acne to cysts or nodules\n• Face, neck, upper chest, and back may be affected\n• Pigmentary changes and severe scarring can occur\n■ Differential Diagnosis\n• Acne rosacea, perioral dermatitis, gram-negative follicu

In [1]:
import os
import time
import google.generativeai as genai
import pinecone
from pinecone import Pinecone
from langchain.vectorstores import Pinecone as PineconeVectorStore
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document

# 🔹 API Keys (Set these before running)
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
INDEX_NAME = "medicalbot"  # Change this to your actual Pinecone index name

# 🔹 Initialize Gemini AI Model
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-pro")  # Gemini AI Model

# 🔹 Initialize Pinecone (New Method)
if not PINECONE_API_KEY:
    raise ValueError("PINECONE_API_KEY is missing. Set it in environment variables.")

# ✅ Use Pinecone class to initialize
pc = Pinecone(api_key=PINECONE_API_KEY)

# 🔹 Check if the index exists
if INDEX_NAME not in [index_info['name'] for index_info in pc.list_indexes()]:
    raise ValueError(f"Index '{INDEX_NAME}' not found in Pinecone. Check your Pinecone console.")

# 🔹 Load Pinecone Retriever
retriever = PineconeVectorStore.from_existing_index(
    INDEX_NAME,
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
)
print("✅ Retriever loaded successfully!")

# 🔹 Function to Handle API Rate Limits
def generate_response_with_retry(prompt, max_retries=3, delay=5):
    """Handles Gemini AI API rate limits with retry logic."""
    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt)
            return response.text  # Extract text from response
        except genai.RateLimitError:
            print(f"⚠ Rate limit reached. Retrying in {delay} seconds... (Attempt {attempt + 1})")
            time.sleep(delay)
    return "❌ Error: Rate limit exceeded. Please try again later."

# 🔹 Custom RAG Query Function
def custom_rag_chain(query):
    """Retrieves relevant documents and generates an answer."""
    retrieved_docs = retriever.as_retriever().invoke(query)  # Retrieve from Pinecone

    if retrieved_docs:  # Use retrieved knowledge
        retrieved_text = "\n\n".join([doc.page_content for doc in retrieved_docs])
        prompt = f"Based on the following medical references, answer the question:\n\n{retrieved_text}\n\nQuestion: {query}"
        return generate_response_with_retry(prompt)
    
    else:  # Fall back to general Gemini AI knowledge
        print("⚠ No relevant documents found. Answering from general knowledge...")
        return generate_response_with_retry(query)

# 🔹 Example Query
if __name__ == "__main__":
    query = "What is Atrial Flutter?"  # Example medical question
    response = custom_rag_chain(query)
    print("\n📝 *Response:*", response)

d:\chatbot\medicalchatbot\medibot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\gaswi\AppData\Local\Temp\ipykernel_9852\3341281340.py:33: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


✅ Retriever loaded successfully!

📝 *Response:* Atrial flutter is a common heart rhythm disorder characterized by rapid electrical impulses in the atria (the upper chambers of the heart) which typically occur at a rate between 250 and 350 beats per minute with every second, third, or fourth impulse conducted by the ventricle (lower chambers of the heart). It is commonly seen in chronic obstructive pulmonary disease (COPD) and dilated cardiomyopathy, especially in alcoholics. Patients may experience symptoms such as palpitations or evidence of congestive heart failure, but some may remain asymptomatic. In occasional patients, flutter (a) waves may be visible in the neck.
